In [1]:
%cd /mnt/c/Users/user/Desktop/과제/binfo1-work

/mnt/c/Users/user/Desktop/과제/binfo1-work


/root/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [125]:
!pip install pandas numpy

In [2]:
!grep -i mirlet7d ../binfo1-datapack1/gencode.gtf

chr13	ENSEMBL	gene	48689488	48689590	.	-	.	gene_id "ENSMUSG00000065453.3"; gene_type "miRNA"; gene_name "Mirlet7d"; level 3; mgi_id "MGI:2676796";
chr13	ENSEMBL	transcript	48689488	48689590	.	-	.	gene_id "ENSMUSG00000065453.3"; transcript_id "ENSMUST00000083519.3"; gene_type "miRNA"; gene_name "Mirlet7d"; transcript_type "miRNA"; transcript_name "Mirlet7d-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676796"; tag "basic";
chr13	ENSEMBL	exon	48689488	48689590	.	-	.	gene_id "ENSMUSG00000065453.3"; transcript_id "ENSMUST00000083519.3"; gene_type "miRNA"; gene_name "Mirlet7d"; transcript_type "miRNA"; transcript_name "Mirlet7d-201"; exon_number 1; exon_id "ENSMUSE00000522678.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676796"; tag "basic";


In [61]:
!samtools view -b -o CLIP-let7d.bam ../binfo1-datapack1/CLIP-35L33G.bam chr13:48689488-48689590
!samtools view CLIP-let7d.bam | wc -l

187


In [62]:
!samtools mpileup CLIP-let7d.bam > CLIP-let7d.pileup
!wc -l CLIP-let7d.pileup

[mpileup] 1 samples in 1 input files
87 CLIP-let7d.pileup


In [63]:
!head CLIP-let7d.pileup

chr13	48689488	N	24	^Ga^Ia^Ha^Ia^Ha^Ia^Ia^Ia^Ga^Ia^Ia^Ia^Ia^Ia^Ha^Ia^Ga^Ga^Ga^Ia^Ea^Da^Ha^Ga	BIHGGIHIEIIIGIHIGEGH5BDD
chr13	48689489	N	53	aaaaaaaaaaaaaaaaaaaaaaaaa^Ia^Ha^Ia^Ga^Ga^Ha^Ia^Ia^Ga^Ga^Ia^Ia^Ha^Ia^Ia^Ha^Ha^Da^Ia^Ha^Ia^Ia^Ia^Ia^Ia^Ha^Ga^Ga	GIHIGIIIEIIIIIHIG>EB4@5HDIHIBEGIID?IIGII@H>IGHIIIGHDC
chr13	48689490	N	69	ttttttttttttttttttttttttttttttttttttttttttttttttttttt^Gt^Gt^Ht^Gt^It^It^Ht^It^Gt^Gt^Ht^It^It^It^Ht^It	GIHIGHIFFIIIIIFHGEDG;2>FDIHIDGBIIDBIIDIIAH>IBGIIIGHBG=BG=IIGIDGHFIIHI
chr13	48689491	N	105	aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa^Ia^Ia^Ha^Ga^Ia^Ga^Ia^Ga^Ha^Ha^Ha^Ga^Ia^Ha^Ha^Ia^@a^Ia^Ia^Ia^Ga^Ha^Ha^Ga^Ha^Ga^Ia^Ha^Ga^Ia^Ga^Ha^Ga^Ia^Ia^Ga	EIGHEHHIEIIIIIHIGCBH:?>HGGHI>GBIIGBIIGIHBB;IHIIIIGHDC3=G:IGGIDEHIFIGHII=GIDHEGHHEIGGI7III9=HGBAIGBIGFG>I2
chr13	48689492	N	105	aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa	GIGIEIEI7IIIGIGIG;?F:D@G7GHIAGAIIG3IIGIHDB2IHIIIIDDGC1<GCIIHIGCHIIIEIIIGCI

In [64]:
!awk '$2 >= 48689488 && $2 <= 48689590 { print $0; }' CLIP-let7d.pileup > CLIP-let7d-gene.pileup
!tail CLIP-let7d-gene.pileup

chr13	48689565	N	6	cccccc	>IGIIH
chr13	48689566	N	6	tttttt	:IGFIH
chr13	48689567	N	6	aaaaaa	8IFIFI
chr13	48689568	N	6	cccccc	?IGIII
chr13	48689569	N	6	tttttt	;IGIII
chr13	48689570	N	5	aaaaa	EDECD
chr13	48689571	N	6	cccccc	:GGIGG
chr13	48689572	N	6	cccccc	:GGIGG
chr13	48689573	N	6	tttttt	5GGIGG
chr13	48689574	N	6	c$c$c$c$c$c$	;GGIGG


In [65]:
import pandas as pd
pileup = pd.read_csv('CLIP-let7d-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

,chrom,pos,_ref,count,basereads,quals
82,chr13,48689570,N,5,aaaaa,EDECD
83,chr13,48689571,N,6,cccccc,:GGIGG
84,chr13,48689572,N,6,cccccc,:GGIGG
85,chr13,48689573,N,6,tttttt,5GGIGG
86,chr13,48689574,N,6,c$c$c$c$c$c$,;GGIGG


In [66]:
import re
toremove = re.compile('[<>$*#^]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

In [67]:
import math
from collections import Counter

def shannon_entropy(data):
    counts = Counter(data)
    total = len(data)
    entropy = -sum(count/total * math.log2(count/total) for count in counts.values())
    return entropy

In [68]:
pileup.loc[:, 'Shannon entropy'] = pileup['matches'].apply(shannon_entropy)
pileup

,chrom,pos,_ref,count,basereads,quals,matches,Shannon entropy
0,chr13,48689488,N,24,^Ga^Ia^Ha^Ia^Ha^Ia^Ia^Ia^Ga^Ia^Ia^Ia^Ia^Ia^Ha^...,BIHGGIHIEIIIGIHIGEGH5BDD,GaIaHaIaHaIaIaIaGaIaIaIaIaIaHaIaGaGaGaIaEaDaHaGa,1.906454
1,chr13,48689489,N,53,aaaaaaaaaaaaaaaaaaaaaaaaa^Ia^Ha^Ia^Ga^Ga^Ha^Ia...,GIHIGIIIEIIIIIHIG>EB4@5HDIHIBEGIID?IIGII@H>IGH...,aaaaaaaaaaaaaaaaaaaaaaaaaIaHaIaGaGaHaIaIaGaGaI...,1.499800
2,chr13,48689490,N,69,tttttttttttttttttttttttttttttttttttttttttttttt...,GIHIGHIFFIIIIIFHGEDG;2>FDIHIDGBIIDBIIDIIAH>IBG...,tttttttttttttttttttttttttttttttttttttttttttttt...,0.988811
3,chr13,48689491,N,105,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,EIGHEHHIEIIIIIHIGCBH:?>HGGHI>GBIIGBIIGIHBB;IHI...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,1.258637
4,chr13,48689492,N,105,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,GIGIEIEI7IIIGIGIG;?F:D@G7GHIAGAIIG3IIGIHDB2IHI...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,-0.000000
...,...,...,...,...,...,...,...,...
82,chr13,48689570,N,5,aaaaa,EDECD,aaaaa,-0.000000
83,chr13,48689571,N,6,cccccc,:GGIGG,cccccc,-0.000000
84,chr13,48689572,N,6,cccccc,:GGIGG,cccccc,-0.000000
85,chr13,48689573,N,6,tttttt,5GGIGG,tttttt,-0.000000


In [69]:
selected_column = pileup['matches']
mask = selected_column.str.match('^[ATGCatgc]*$')
f_pileup = pileup[mask]
f_pileup

,chrom,pos,_ref,count,basereads,quals,matches,Shannon entropy
4,chr13,48689492,N,105,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,GIGIEIEI7IIIGIGIG;?F:D@G7GHIAGAIIG3IIGIHDB2IHI...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,-0.0
5,chr13,48689493,N,105,gggggggggggggggggggggggggggggggggggggggggggggg...,BIEIGHGI7IHHGIGHG=?F58>E5EHIAGAHIDAHIGII:EDHHI...,gggggggggggggggggggggggggggggggggggggggggggggg...,-0.0
6,chr13,48689494,N,105,gggggggggggggggggggggggggggggggggggggggggggggg...,FIFEGIFE3HFIGIGHG=?D1?@D5EHIAEADIBFIIGHIBD<IHD...,gggggggggggggggggggggggggggggggggggggggggggggg...,-0.0
7,chr13,48689495,N,104,cccccccccccccccccccccccccccccccccccccccccccccc...,GID>GHEG.HIHGH>IG2;B;?>E0BCI8G?FGA?HH@HIB?DHBI...,cccccccccccccccccccccccccccccccccccccccccccccc...,-0.0
8,chr13,48689496,N,102,cccccccccccccccccccccccccccccccccccccccccccccc...,FI8EEGEI.GEB<ID>G38B19>BFFI<D<FGAEFIGDFB@@CH>E...,cccccccccccccccccccccccccccccccccccccccccccccc...,-0.0
...,...,...,...,...,...,...,...,...
82,chr13,48689570,N,5,aaaaa,EDECD,aaaaa,-0.0
83,chr13,48689571,N,6,cccccc,:GGIGG,cccccc,-0.0
84,chr13,48689572,N,6,cccccc,:GGIGG,cccccc,-0.0
85,chr13,48689573,N,6,tttttt,5GGIGG,tttttt,-0.0


In [70]:
f_pileup1 = f_pileup[f_pileup['Shannon entropy'] != 0]
f_pileup1

,chrom,pos,_ref,count,basereads,quals,matches,Shannon entropy
22,chr13,48689510,N,182,cccccccccccccccccccccccccccccccccccccccccccccc...,BIGDGHD>H>FIIIEHIG@BI86BH?DHIB>DII6EHIHHIGB8IF...,cccccccccccccccccccccccccccccccccccccccccccccc...,0.049157
32,chr13,48689520,N,187,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaagaaaaaaa...,GIHIDGHBIGIIIIIGIGDGI8;AHGHHIG@GII@AII3IIHGDHG...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaagaaaaaaa...,0.048052
37,chr13,48689525,N,186,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,CIGIGII<I?IIIIIGIGDBH=@@DDIFI>/GII@BIIHIIHDIGH...,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,0.048269
40,chr13,48689528,N,186,*c*cc*cc*ccccccc*cc*cc*ccc**cc*c**cc*cccc**c*c...,GIBGHII>I?HGGIGGIGGEH7<>HBFHIBEGEHGDIIHIIHG/IG...,cccccccccccccccccccccccccccccccccccccccccccccc...,0.851251
41,chr13,48689529,N,187,cccttcctccttttctcctctccctccctccccctccttttcctcc...,GIBIHII<I<IIIIIGIGGEI5<>GBFHIAEGEHDBIIHIIHG5IE...,cccttcctccttttctcctctccctccctccccctccttttcctcc...,1.450881
43,chr13,48689531,N,174,ttttttttttttttttttttttttttttt$ttttttttt$t$t$tt...,GIHIHIFDHBHIHIIGIGG<I7G?HBIGGEEBIICDIIHIIHG?IG...,tttttttttttttttttttttttttttttttttttttttttttttt...,0.051043
54,chr13,48689542,N,106,tttttttttttttttttttttttttttttttttttttttttttttt...,BIHHFIG1IGIIIIIFH7:>I7G4H?IEBBBIIGFIIHBI>IIB<8...,tttttttttttttttttttttttttttttttttttttttttttttt...,0.077017


In [71]:
import pysam

# bam 파일 열기
bam_file = pysam.AlignmentFile("CLIP-let7d.bam", "rb")

# 시퀀스를 저장할 리스트 생성
sequences = []
s_positions = []
e_positions = []

# bam 파일에서 시퀀스 추출
for read in bam_file:
    sequence = read.query_sequence
    sequences.append(sequence)
    start_position = read.reference_start
    s_positions.append(start_position)
    end_pos = read.reference_end
    e_positions.append(end_pos)


# 추출된 시퀀스 출력 또는 다른 작업 수행
#for sequence in sequences:
    #for position in positions:
        #print(sequence, position)

# bam 파일 닫기
bam_file.close()


In [72]:
df = pd.DataFrame({'chr': 'chr13', 'start position':s_positions, 'end position':e_positions, 'sequence':sequences})

df

,chr,start position,end position,sequence
0,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTGTG...
1,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCCTTGT...
2,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTGTG...
3,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTTGT...
4,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTTGT...
...,...,...,...,...
182,chr13,48689502,48689551,AAGGCAGCAGGTCGTATAGTTAACTCCTTGTGGGCAAAATCTCTGCCCT
183,chr13,48689502,48689551,AAGGCAGCAGGTCGTATAGTTAACTCTTTGTGGGCAAAATCTCTGCCCT
184,chr13,48689502,48689551,AAGGCAGCAGGTCGTATAGTTAACTCTTTGTGGGCAAAATCTCTGCCCT
185,chr13,48689502,48689551,AAGGCAGCAGGTCGTATAGTTAACTCTTTGTGGGCAAAATCTCTGCCCT


In [113]:
result = df.groupby(['chr', 'start position', 'end position', 'sequence']).size().reset_index(name='Count')
result

,chr,start position,end position,sequence,Count
0,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCCTTGT...,8
1,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTGTG...,7
2,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTTGT...,11
3,chr13,48689488,48689531,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTT,7
4,chr13,48689488,48689531,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATGGTTAACTCTTT,1
5,chr13,48689488,48689550,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCCTTGTG...,5
6,chr13,48689488,48689551,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTGTGG...,11
7,chr13,48689488,48689551,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTTGTG...,4
8,chr13,48689489,48689551,TAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTTGTGG...,16
9,chr13,48689490,48689530,AAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCCT,3


In [60]:
with open('Mirlet7d.bed', 'w') as f:
    for index, row in result.iterrows():
        chromosome_name = row['chr']
        start_position = row['start position']
        end_position = row['end position']
        readcount = row['Count']
        f.write(f"{chromosome_name}\t{start_position}\t{end_position}\t{readcount}\n")

In [89]:
ref_seq = 'AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCCTTGTGGGCAAAATCTCTGCCCTAAAACTATGCAACCTACTACCTCTTCC'
ref_seq_start = 48689487

In [90]:
def Make_Df(seq, start):
    sequence = []
    pos = []
    for i in range(len(seq)):
        sequence.append(seq[i])
        pos.append(start + i)

    df = pd.DataFrame({'base': sequence, 'pos':pos})

    return df

In [91]:
Ref_df = Make_Df(ref_seq, ref_seq_start)
Ref_df

,base,pos
0,A,48689487
1,A,48689488
2,T,48689489
3,A,48689490
4,A,48689491
...,...,...
86,C,48689573
87,T,48689574
88,T,48689575
89,C,48689576


In [94]:
x = Make_Df(result['sequence'][0], result['start position'][0])
f_data = pd.merge(Ref_df, x, left_on='pos', right_on='pos')
f_data

,base_x,pos,base_y
0,A,48689487,A
1,A,48689488,A
2,T,48689489,T
3,A,48689490,A
4,A,48689491,A
...,...,...,...
59,G,48689546,G
60,C,48689547,C
61,C,48689548,C
62,C,48689549,C


In [98]:
#Mutations: column no., position, Ref_base, read_base
Mutations = []
for i in range(len(result)):
    x = Make_Df(result['sequence'][i], result['start position'][i])
    #print(len(x))
    f_data = pd.merge(Ref_df, x, left_on='pos', right_on='pos')
    #print(len(f_data))
    for j in range(len(f_data)):
        a = f_data['base_x'][j]
        b = f_data['base_y'][j]
        if f_data['base_x'][j] != f_data['base_y'][j]:
            Mutations.append([i, pos[j], a, b])
            #print(i, j, f_data['base_x'][j], f_data['base_y'][j])

print(Mutations)

[[1, 48689528, 'C', 'T'], [1, 48689530, 'T', 'G'], [1, 48689531, 'G', 'T'], [1, 48689532, 'T', 'G'], [1, 48689535, 'G', 'C'], [1, 48689536, 'C', 'A'], [1, 48689540, 'A', 'T'], [1, 48689541, 'T', 'C'], [1, 48689542, 'C', 'T'], [1, 48689543, 'T', 'C'], [1, 48689544, 'C', 'T'], [1, 48689545, 'T', 'G'], [1, 48689546, 'G', 'C'], [1, 48689549, 'C', 'T'], [2, 48689528, 'C', 'T'], [3, 48689527, 'C', 'T'], [4, 48689518, 'A', 'G'], [4, 48689527, 'C', 'T'], [6, 48689527, 'C', 'T'], [6, 48689529, 'T', 'G'], [6, 48689530, 'G', 'T'], [6, 48689531, 'T', 'G'], [6, 48689534, 'G', 'C'], [6, 48689535, 'C', 'A'], [6, 48689539, 'A', 'T'], [6, 48689540, 'T', 'C'], [6, 48689541, 'C', 'T'], [6, 48689542, 'T', 'C'], [6, 48689543, 'C', 'T'], [6, 48689544, 'T', 'G'], [6, 48689545, 'G', 'C'], [6, 48689548, 'C', 'T'], [7, 48689527, 'C', 'T'], [8, 48689526, 'C', 'T'], [10, 48689525, 'C', 'T'], [12, 48689524, 'C', 'G'], [13, 48689525, 'C', 'G'], [13, 48689547, 'T', 'N'], [14, 48689525, 'C', 'G'], [15, 48689525, 'C',

In [117]:
M_df = pd.DataFrame(Mutations)
M_df.columns = ['column', 'pos', 'Ref', 'read']

M_df

,column,pos,Ref,read
0,1,48689528,C,T
1,1,48689530,T,G
2,1,48689531,G,T
3,1,48689532,T,G
4,1,48689535,G,C
...,...,...,...,...
63,24,48689513,C,G
64,26,48689512,C,G
65,28,48689513,C,T
66,29,48689509,A,C


In [179]:
counts = M_df['column'].value_counts()

# 3번 이상 나타나는 값을 찾기
to_remove = counts[counts > 5].index

# 이 값을 포함하는 행 제거
df_filtered = M_df[~M_df['column'].isin(to_remove)]

df_filtered = df_filtered[df_filtered['read'] != 'N']
df_filtered

,column,pos,Ref,read
14,2,48689528,C,T
15,3,48689527,C,T
16,4,48689518,A,G
17,4,48689527,C,T
32,7,48689527,C,T
33,8,48689526,C,T
34,10,48689525,C,T
35,12,48689524,C,G
36,13,48689525,C,G
38,14,48689525,C,G


In [191]:
import numpy as np
result['Mutation'] = np.nan

In [192]:
def append_data(row, new_data):
    if pd.isna(row):
        return new_data
    else:
        return f"{row}_{new_data}"

In [193]:
for i in range(len(df_filtered)):
    lo = df_filtered['column'].iloc[i]
    data = f"{df_filtered['pos'].iloc[i]}_{df_filtered['Ref'].iloc[i]}_{df_filtered['read'].iloc[i]}"
    result.loc[lo, 'Mutation'] = append_data(result.loc[lo, 'Mutation'], data)

/tmp/ipykernel_2150/1376914472.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '48689528_C_T' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result.loc[lo, 'Mutation'] = append_data(result.loc[lo, 'Mutation'], data)


In [194]:
result

,chr,start position,end position,sequence,Count,Mutation
0,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCCTTGT...,8,NaN
1,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTGTG...,7,NaN
2,chr13,48689487,48689551,AATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTTGT...,11,48689528_C_T
3,chr13,48689488,48689531,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTT,7,48689527_C_T
4,chr13,48689488,48689531,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATGGTTAACTCTTT,1,48689518_A_G_48689527_C_T
5,chr13,48689488,48689550,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCCTTGTG...,5,NaN
6,chr13,48689488,48689551,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTGTGG...,11,NaN
7,chr13,48689488,48689551,ATAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTTGTG...,4,48689527_C_T
8,chr13,48689489,48689551,TAAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCTTTGTGG...,16,48689526_C_T
9,chr13,48689490,48689530,AAGGCCCTAAGAAAGGCAGCAGGTCGTATAGTTAACTCCT,3,NaN


In [195]:
with open('Mirlet7d.bed', 'w') as f:
    for index, row in result.iterrows():
        chromosome_name = row['chr']
        start_position = row['start position']
        end_position = row['end position']
        readcount = row['Count']
        mutation = row['Mutation']
        if pd.isna(mutation):
            f.write(f"{chromosome_name}\t{start_position}\t{end_position}\t{readcount}_No_mutation\n")
        else:
            f.write(f"{chromosome_name}\t{start_position}\t{end_position}\t{readcount}_{mutation}\n")


In [196]:
pd.read_csv('Mirlet7d.bed', sep='\t')

,chr13,48689487,48689551,8_No_mutation
0,chr13,48689487,48689551,7_No_mutation
1,chr13,48689487,48689551,11_48689528_C_T
2,chr13,48689488,48689531,7_48689527_C_T
3,chr13,48689488,48689531,1_48689518_A_G_48689527_C_T
4,chr13,48689488,48689550,5_No_mutation
5,chr13,48689488,48689551,11_No_mutation
6,chr13,48689488,48689551,4_48689527_C_T
7,chr13,48689489,48689551,16_48689526_C_T
8,chr13,48689490,48689530,3_No_mutation
9,chr13,48689490,48689530,6_48689525_C_T
